# Descriptive statistic comparison
### By Jack Demeter, Jacob Laframboise, Daniella Ruisendaal, Jesse MacCormac
Here we take in a dataset of imu data recorded from the TurtleBot3 Waffle Pi driving on 2 different surfaces at 4 different speeds each, and use descriptive statistics to compare and examine for patterns. 

In [38]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer

import plotly as ply
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
%matplotlib notebook

pd.set_option('display.max_columns', 120)
pd.set_option('display.max_rows', 80)

## Data Pre-Processing
#### We load in individual csv files collected from Rosbag on TurtleBot3 with ROS.

In [39]:
dataFolder = r"C:\Users\jaker\Documents\Experiment3Data-2019-11-21"

dataFiles = [
    r"gMitTile_s15_t8.csv",
    r"gMitTile_s15_t9.csv",
    r"gMitTile_s15_t10.csv",
    r"gTurf_s15_t3.csv",
    r"gTurf_s15_t4.csv",
    r"gTurf_s15_t5.csv",
    r"gTurf_s15_t6.csv",
    r"gTurf_s15_t7.csv",
    r"gTurf_s15_t8.csv",
    r"gTurf_s15_t9.csv",
    r"gTurf_s15_t10.csv",
    r"gArcTile_s15_t3.csv",
    r"gArcTile_s15_t4.csv",
    r"gArcTile_s15_t5.csv",
    r"gArcTile_s15_t6.csv",
    r"gArcTile_s15_t7.csv",
    r"gArcTile_s15_t8.csv",
    r"gArcTile_s15_t9.csv",
    r"gArcTile_s15_t10.csv",
    r"gCarp_s15_t3.csv",
    r"gCarp_s15_t4.csv",
    r"gCarp_s15_t5.csv",
    r"gCarp_s15_t6.csv",
    r"gCarp_s15_t7.csv",
    r"gCarp_s15_t8.csv",
    r"gCarp_s15_t9.csv",
    r"gCarp_s15_t10.csv",
    r"gMitTile_s15_t3.csv",
    r"gMitTile_s15_t4.csv",
    r"gMitTile_s15_t5.csv",
    r"gMitTile_s15_t6.csv",
    r"gMitTile_s15_t7.csv"
]

savePath = "Data-32Series-Delta30-Squared.csv"

In [40]:
""" 
For each data file we:
adjust the index, 
interpolate NaN values,
drop remaining NaN values, 
drop some empty columns.

We then augment the feature space with delta columns, 
and with polynomial columns,
and label the columns with terrain, speed, and trial number
"""
for i in range(len(dataFiles)):
    terrain = dataFiles[i].split('_')[0][1:]
    speed = dataFiles[i].split('_s')[1][:2]
    trial = dataFiles[i].split('_t')[1][0]

    df = pd.read_csv(os.path.join(dataFolder, dataFiles[i]))
    df = df.rename(columns={'Unnamed: 0': 'Seq'})
    df = df.set_index('Seq')

    #print(df.isnull().sum().sum())
    df = df.interpolate(method='polynomial', order=1)
    #print(df.isnull().sum().sum())

    df = df.dropna()
    #print(df.isnull().sum().sum())
    df = df.reset_index().drop(columns=['Seq'])
    
    df['OdomPosX'] = df['OdomPosX'] - df['OdomPosX'].loc[0]
    df['OdomPosY'] = df['OdomPosY'] - df['OdomPosY'].loc[0]

    df = df.drop(columns=['OdomPosZ', 'OdomOrientX', 'OdomOrientY', 'OdomLinY', 'OdomLinZ', 'OdomAngX', 'OdomAngY'])
    df['OdomPosXY'] = np.sqrt(df.OdomPosX**2 + df.OdomPosY**2)
    df = df.drop(columns=['OdomPosY', 'OdomPosX'])
    
    for c in df.columns.tolist():
        print(c)
    
    order=2
    
    dList = range(1, 302, 20)
    # dList = [2,4,8,16,32,64]
    for col in df.columns.tolist():
        if col!='Sensor':
            for d in dList:
                df[col+'Delta{}'.format(d)] = df[col].diff(d)
                if order>1:
                    for p in range(2, order+1):
                        df[col+'Delta{}Exp{}'.format(d, p)] = df[col+'Delta{}'.format(d)]**p
                        #print('Added ' + col +'Delta{}'.format(d))
        else:
            pass

    df = df.iloc[max(dList):].reset_index().drop(columns=['index'])
    df = df.drop(columns=['Sensor', 'Time'])
    df['Speed']=int(speed)
    df['Terrain']=terrain
    df['Trial']=int(trial)

    if i==0:
        mainDf = df.copy(deep=True)
    else:
        mainDf = pd.concat([mainDf, df], axis=0, sort=False)
    print("Added {} of size {} to mainDf. ".format(dataFiles[i], df.shape))
    print("Data series completed: {}/{}".format(i+1,len(dataFiles)))
    print("MainDf is now size {}".format(mainDf.shape))
    print('\n')

Time
OdomOrientZ
OdomOrientW
OdomLinX
OdomAngZ
Sensor
ImuOrientX
ImuOrientY
ImuOrientZ
ImuOrientW
ImuAngVelX
ImuAngVelY
ImuAngVelZ
ImuAccelX
ImuAccelY
ImuAccelZ
OdomPosXY
Added gMitTile_s15_t8.csv of size (3530, 530) to mainDf. 
Data series completed: 1/32
MainDf is now size (3530, 530)


Time
OdomOrientZ
OdomOrientW
OdomLinX
OdomAngZ
Sensor
ImuOrientX
ImuOrientY
ImuOrientZ
ImuOrientW
ImuAngVelX
ImuAngVelY
ImuAngVelZ
ImuAccelX
ImuAccelY
ImuAccelZ
OdomPosXY
Added gMitTile_s15_t9.csv of size (4030, 530) to mainDf. 
Data series completed: 2/32
MainDf is now size (7560, 530)


Time
OdomOrientZ
OdomOrientW
OdomLinX
OdomAngZ
Sensor
ImuOrientX
ImuOrientY
ImuOrientZ
ImuOrientW
ImuAngVelX
ImuAngVelY
ImuAngVelZ
ImuAccelX
ImuAccelY
ImuAccelZ
OdomPosXY
Added gMitTile_s15_t10.csv of size (4552, 530) to mainDf. 
Data series completed: 3/32
MainDf is now size (12112, 530)


Time
OdomOrientZ
OdomOrientW
OdomLinX
OdomAngZ
Sensor
ImuOrientX
ImuOrientY
ImuOrientZ
ImuOrientW
ImuAngVelX
ImuAngVelY
ImuAngVe

Added gMitTile_s15_t4.csv of size (4423, 530) to mainDf. 
Data series completed: 29/32
MainDf is now size (133485, 530)


Time
OdomOrientZ
OdomOrientW
OdomLinX
OdomAngZ
Sensor
ImuOrientX
ImuOrientY
ImuOrientZ
ImuOrientW
ImuAngVelX
ImuAngVelY
ImuAngVelZ
ImuAccelX
ImuAccelY
ImuAccelZ
OdomPosXY
Added gMitTile_s15_t5.csv of size (3578, 530) to mainDf. 
Data series completed: 30/32
MainDf is now size (137063, 530)


Time
OdomOrientZ
OdomOrientW
OdomLinX
OdomAngZ
Sensor
ImuOrientX
ImuOrientY
ImuOrientZ
ImuOrientW
ImuAngVelX
ImuAngVelY
ImuAngVelZ
ImuAccelX
ImuAccelY
ImuAccelZ
OdomPosXY
Added gMitTile_s15_t6.csv of size (4407, 530) to mainDf. 
Data series completed: 31/32
MainDf is now size (141470, 530)


Time
OdomOrientZ
OdomOrientW
OdomLinX
OdomAngZ
Sensor
ImuOrientX
ImuOrientY
ImuOrientZ
ImuOrientW
ImuAngVelX
ImuAngVelY
ImuAngVelZ
ImuAccelX
ImuAccelY
ImuAccelZ
OdomPosXY
Added gMitTile_s15_t7.csv of size (3883, 530) to mainDf. 
Data series completed: 32/32
MainDf is now size (145353, 530)



In [41]:
# avoid reloading dataset when working
df = mainDf.copy(deep=True)

In [42]:
df.head(10)

,OdomOrientZ,OdomOrientW,OdomLinX,OdomAngZ,ImuOrientX,ImuOrientY,ImuOrientZ,ImuOrientW,ImuAngVelX,ImuAngVelY,ImuAngVelZ,ImuAccelX,ImuAccelY,ImuAccelZ,OdomPosXY,TimeDelta1,TimeDelta1Exp2,TimeDelta21,TimeDelta21Exp2,TimeDelta41,TimeDelta41Exp2,TimeDelta61,TimeDelta61Exp2,TimeDelta81,TimeDelta81Exp2,TimeDelta101,TimeDelta101Exp2,TimeDelta121,TimeDelta121Exp2,TimeDelta141,TimeDelta141Exp2,TimeDelta161,TimeDelta161Exp2,TimeDelta181,TimeDelta181Exp2,TimeDelta201,TimeDelta201Exp2,TimeDelta221,TimeDelta221Exp2,TimeDelta241,TimeDelta241Exp2,TimeDelta261,TimeDelta261Exp2,TimeDelta281,TimeDelta281Exp2,TimeDelta301,TimeDelta301Exp2,OdomOrientZDelta1,OdomOrientZDelta1Exp2,OdomOrientZDelta21,OdomOrientZDelta21Exp2,OdomOrientZDelta41,OdomOrientZDelta41Exp2,OdomOrientZDelta61,OdomOrientZDelta61Exp2,OdomOrientZDelta81,OdomOrientZDelta81Exp2,OdomOrientZDelta101,OdomOrientZDelta101Exp2,OdomOrientZDelta121,...,ImuAccelZDelta61Exp2,ImuAccelZDelta81,ImuAccelZDelta81Exp2,ImuAccelZDelta101,ImuAccelZDelta101Exp2,ImuAccelZDelta121,ImuAccelZDelta121Exp2,ImuAccelZDelta141,ImuAccelZDelta141Exp2,ImuAccelZDelta161,ImuAccelZDelta161Exp2,ImuAccelZDelta181,ImuAccelZDelta181Exp2,ImuAccelZDelta201,ImuAccelZDelta201Exp2,ImuAccelZDelta221,ImuAccelZDelta221Exp2,ImuAccelZDelta241,ImuAccelZDelta241Exp2,ImuAccelZDelta261,ImuAccelZDelta261Exp2,ImuAccelZDelta281,ImuAccelZDelta281Exp2,ImuAccelZDelta301,ImuAccelZDelta301Exp2,OdomPosXYDelta1,OdomPosXYDelta1Exp2,OdomPosXYDelta21,OdomPosXYDelta21Exp2,OdomPosXYDelta41,OdomPosXYDelta41Exp2,OdomPosXYDelta61,OdomPosXYDelta61Exp2,OdomPosXYDelta81,OdomPosXYDelta81Exp2,OdomPosXYDelta101,OdomPosXYDelta101Exp2,OdomPosXYDelta121,OdomPosXYDelta121Exp2,OdomPosXYDelta141,OdomPosXYDelta141Exp2,OdomPosXYDelta161,OdomPosXYDelta161Exp2,OdomPosXYDelta181,OdomPosXYDelta181Exp2,OdomPosXYDelta201,OdomPosXYDelta201Exp2,OdomPosXYDelta221,OdomPosXYDelta221Exp2,OdomPosXYDelta241,OdomPosXYDelta241Exp2,OdomPosXYDelta261,OdomPosXYDelta261Exp2,OdomPosXYDelta281,OdomPosXYDelta281Exp2,OdomPosXYDelta301,OdomPosXYDelta301Exp2,Speed,Terrain,Trial
0,0.095387,0.995440,0.148886,-0.006150,-0.004592,-0.004974,-0.104227,-0.994526,0.000000,-0.027669,-0.013835,-0.290297,-0.049081,9.154829,0.280468,0.005092,0.000026,0.130681,0.017078,0.251098,0.063050,0.368369,0.135695,0.493427,0.243470,0.636363,0.404958,0.771192,0.594737,0.892386,0.796352,1.017193,1.034682,1.138310,1.295749,1.258815,1.584615,1.381064,1.907338,1.498555,2.245666,1.625103,2.640961,1.736605,3.015797,1.860827,3.462679,-0.000021,4.593532e-10,-0.000153,2.334641e-08,-0.000206,4.228962e-08,0.000041,1.700049e-09,0.000260,6.770568e-08,0.000506,2.555428e-07,0.000994,...,0.319260,-1.777096,3.158069,0.223260,0.049845,-2.354098,5.541779,-3.133411,9.818267,-1.962646,3.851981,-1.224035,1.498262,-1.312022,1.721402,-0.547673,0.299946,-0.365115,0.133309,0.575208,0.330864,-1.317110,1.734778,-1.399111,1.957512,0.000633,4.004000e-07,0.020527,0.000421,0.037735,0.001424,0.055174,0.003044,0.075799,0.005745,0.097305,0.009468,0.114277,0.013059,0.132897,0.017662,0.153256,0.023487,0.171651,0.029464,0.189526,0.035920,0.207673,0.043128,0.225261,0.050742,0.242993,0.059045,0.259105,0.067135,0.280468,0.078662,15,MitTile,8
1,0.095366,0.995442,0.148886,-0.007478,-0.003933,-0.004679,-0.104174,-0.994536,0.008514,-0.064916,-0.013835,-0.099958,-0.174777,10.842142,0.281100,0.005561,0.000031,0.130839,0.017119,0.251223,0.063113,0.368386,0.135708,0.493495,0.243537,0.634416,0.402483,0.770069,0.593006,0.892173,0.795973,1.017388,1.035079,1.137589,1.294109,1.259750,1.586971,1.380945,1.907008,1.498703,2.246110,1.621436,2.629054,1.735753,3.012839,1.860233,3.460467,-0.000021,4.593532e-10,-0.000181,3.267875e-08,-0.000227,5.162486e-08,-0.000018,3.331765e-10,0.000254,6.460856e-08,0.000447,1.998104e-07,0.000953,...,0.060224,-0.089783,0.008061,1.910573,3.650289,-0.326809,0.106804,0.354342,0.125558,1.449689,2.101598,0.463278,0.214626,0.762553,0.581487,1.201290,1.443098,1.322198,1.748207,1.616685,2.613670,1.389235,1.929975,1.731008,2.996387,0.000633,4.00400

In [43]:
# check for none values and correct size
print(df.isnull().sum().sum()) # check for NaN
df.shape

0


(145353, 530)

### Feature engineering: elimination

In [44]:
# eliminate columns derived from time, and from angular velocity
# columnsToDrop = [x for x in df.columns.tolist() if 'Time' in x or ('Delta' not in x and 'Speed' not in x and 'Terrain' not in x) ]
# df = df.drop(columns=columnsToDrop)

### Speed filtering

In [45]:
df = df[df['Speed']==15]
df.shape

(145353, 530)

### Data segmentation

In [46]:
df_Turf = df[df['Terrain']=='Turf']
df_MitTile = df[df['Terrain']=='MitTile']
df_ArcTile = df[df['Terrain']=='ArcTile']
df_Carp = df[df['Terrain']=='Carp']

In [47]:
df_Turf.mean()

OdomOrientZ              -0.069151
OdomOrientW               0.701928
OdomLinX                  0.149905
OdomAngZ                  0.003330
ImuOrientX               -0.000738
                           ...    
OdomPosXYDelta281Exp2     0.064395
OdomPosXYDelta301         0.271711
OdomPosXYDelta301Exp2     0.073891
Speed                    15.000000
Trial                     5.201462
Length: 529, dtype: float64

In [48]:
df_MitTile.mean()

OdomOrientZ              -0.377383
OdomOrientW               0.688051
OdomLinX                  0.149906
OdomAngZ                  0.001115
ImuOrientX               -0.007379
                           ...    
OdomPosXYDelta281Exp2     0.065172
OdomPosXYDelta301         0.273316
OdomPosXYDelta301Exp2     0.074775
Speed                    15.000000
Trial                     5.186776
Length: 529, dtype: float64

In [49]:
df_ArcTile.mean()

OdomOrientZ               0.355359
OdomOrientW               0.683129
OdomLinX                  0.149915
OdomAngZ                  0.000688
ImuOrientX               -0.002245
                           ...    
OdomPosXYDelta281Exp2     0.065126
OdomPosXYDelta301         0.273205
OdomPosXYDelta301Exp2     0.074725
Speed                    15.000000
Trial                     5.368059
Length: 529, dtype: float64

In [50]:
df_Carp.mean()

OdomOrientZ              -0.177447
OdomOrientW               0.545786
OdomLinX                  0.149939
OdomAngZ                 -0.001403
ImuOrientX               -0.002589
                           ...    
OdomPosXYDelta281Exp2     0.077997
OdomPosXYDelta301         0.281901
OdomPosXYDelta301Exp2     0.088782
Speed                    15.000000
Trial                     5.546636
Length: 529, dtype: float64

In [51]:
df_mean = pd.DataFrame([df_Turf.mean(),df_MitTile.mean(),df_ArcTile.mean(),df_Carp.mean()])

In [52]:
df_mean['Label'] = ['Turf', 'MitTile','ArcTile','Carp']

In [53]:
df_mean.set_index('Label')

,OdomOrientZ,OdomOrientW,OdomLinX,OdomAngZ,ImuOrientX,ImuOrientY,ImuOrientZ,ImuOrientW,ImuAngVelX,ImuAngVelY,ImuAngVelZ,ImuAccelX,ImuAccelY,ImuAccelZ,OdomPosXY,TimeDelta1,TimeDelta1Exp2,TimeDelta21,TimeDelta21Exp2,TimeDelta41,TimeDelta41Exp2,TimeDelta61,TimeDelta61Exp2,TimeDelta81,TimeDelta81Exp2,TimeDelta101,TimeDelta101Exp2,TimeDelta121,TimeDelta121Exp2,TimeDelta141,TimeDelta141Exp2,TimeDelta161,TimeDelta161Exp2,TimeDelta181,TimeDelta181Exp2,TimeDelta201,TimeDelta201Exp2,TimeDelta221,TimeDelta221Exp2,TimeDelta241,TimeDelta241Exp2,TimeDelta261,TimeDelta261Exp2,TimeDelta281,TimeDelta281Exp2,TimeDelta301,TimeDelta301Exp2,OdomOrientZDelta1,OdomOrientZDelta1Exp2,OdomOrientZDelta21,OdomOrientZDelta21Exp2,OdomOrientZDelta41,OdomOrientZDelta41Exp2,OdomOrientZDelta61,OdomOrientZDelta61Exp2,OdomOrientZDelta81,OdomOrientZDelta81Exp2,OdomOrientZDelta101,OdomOrientZDelta101Exp2,OdomOrientZDelta121,...,ImuAccelZDelta61,ImuAccelZDelta61Exp2,ImuAccelZDelta81,ImuAccelZDelta81Exp2,ImuAccelZDelta101,ImuAccelZDelta101Exp2,ImuAccelZDelta121,ImuAccelZDelta121Exp2,ImuAccelZDelta141,ImuAccelZDelta141Exp2,ImuAccelZDelta161,ImuAccelZDelta161Exp2,ImuAccelZDelta181,ImuAccelZDelta181Exp2,ImuAccelZDelta201,ImuAccelZDelta201Exp2,ImuAccelZDelta221,ImuAccelZDelta221Exp2,ImuAccelZDelta241,ImuAccelZDelta241Exp2,ImuAccelZDelta261,ImuAccelZDelta261Exp2,ImuAccelZDelta281,ImuAccelZDelta281Exp2,ImuAccelZDelta301,ImuAccelZDelta301Exp2,OdomPosXYDelta1,OdomPosXYDelta1Exp2,OdomPosXYDelta21,OdomPosXYDelta21Exp2,OdomPosXYDelta41,OdomPosXYDelta41Exp2,OdomPosXYDelta61,OdomPosXYDelta61Exp2,OdomPosXYDelta81,OdomPosXYDelta81Exp2,OdomPosXYDelta101,OdomPosXYDelta101Exp2,OdomPosXYDelta121,OdomPosXYDelta121Exp2,OdomPosXYDelta141,OdomPosXYDelta141Exp2,OdomPosXYDelta161,OdomPosXYDelta161Exp2,OdomPosXYDelta181,OdomPosXYDelta181Exp2,OdomPosXYDelta201,OdomPosXYDelta201Exp2,OdomPosXYDelta221,OdomPosXYDelta221Exp2,OdomPosXYDelta241,OdomPosXYDelta241Exp2,OdomPosXYDelta261,OdomPosXYDelta261Exp2,OdomPosXYDelta281,OdomPosXYDelta281Exp2,OdomPosXYDelta301,OdomPosXYDelta301Exp2,Speed,Trial
Label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Turf,-0.069151,0.701928,0.149905,0.003330,-0.000738,0.003948,0.096302,-0.223438,0.001548,-0.003027,0.003300,0.215028,0.200633,10.316826,2.771013,0.006033,0.000119,0.126715,0.016927,0.247449,0.062396,0.368257,0.136942,0.488920,0.240391,0.609569,0.373045,0.730198,0.534872,0.850852,0.725684,0.971500,0.945747,1.092129,1.194853,1.212770,1.473136,1.333411,1.780475,1.454079,2.116960,1.574738,2.482681,1.695397,2.877350,1.815991,3.301055,0.000008,5.453377e-09,0.000169,6.126675e-07,0.000329,1.673959e-06,0.000488,3.158509e-06,0.000647,5.006448e-06,0.000805,7.142105e-06,0.000962,...,-0.000197,0.214348,-0.000443,0.225010,-0.000563,0.210392,-0.000528,0.219869,-0.000805,0.220730,-0.000827,0.216866,-0.000825,0.219244,-0.000992,0.217086,-0.001077,0.218117,-0.000954,0.214980,-0.000945,0.219642,-0.000913,0.220351,-0.001311,0.214316,0.000901,0.000001,0.018929,0.000375,0.036963,0.001382,0.055012,0.003048,0.073067,0.005361,0.091120,0.008327,0.109173,0.011947,0.127232,0.016220,0.145287,0.021144,0.163343,0.026719,0.181398,0.032949,0.199457,0.039830,0.217520,0.047366,0.235581,0.055554,0.253644,0.064395,0.271711,0.073891,15.0,5.201462
MitTile,-0.377383,0.688051,0.149906,0.001115,-0.007379,0.000646,-0.358435,-0.440394,0.001835,-0.002521,0.001025,0.168531,0.198983,10.334165,2.191231,0.006063,0.000162,0.127354,0.017510,0.248674,0.063204,0.370000,0.138556,0.491289,0.243287,0.612558,0.376919,0.733827,0.540766,0.855064,0.733221,0.976292,0.955645,1.097515,1.207159,1.218738,1.488036,1.339981,1.798638,1.461195,2.138141,1.582443,2.507584,1.703680,2.906069,1.824902,3.334049,0.000002,1.073942e-09,0.000041,1.261227e-07,0.000079,3.201853e-07,0.000117,5.565631e-07,0.000154,8.096740e-07,0.000192,1.047175e-06,0.000230,...,0.000780,2.407341,0.000004,2.597881,0.000227,2.280244,0.000468,2.534933,0.000063,2.431046,-0.000412,2.38